In [ ]:
%pip install jupyterlab opencv-python-headless

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import json

# Muat model yang telah dilatih
model_path = './checkpoints/model_checkpoint2.keras'  # Ganti dengan path lokal ke model Anda
model = load_model(model_path)

# Mendapatkan jumlah kelas dari model
num_classes = model.layers[-1].units
print(f"Jumlah kelas dalam model: {num_classes}")

# Memastikan jumlah kelas sesuai dengan daftar kelas
classes_list_path = './data/classes_list.json'  # Ganti dengan path lokal ke file JSON Anda
with open(classes_list_path, 'r') as f:
    classes_list = json.load(f)

if num_classes == len(classes_list):
    print("Jumlah kelas dalam model sesuai dengan daftar kelas.")
else:
    print(f"Warning: Jumlah kelas dalam model ({num_classes}) tidak sesuai dengan daftar kelas ({len(classes_list)})")

# Fungsi untuk pra-pemrosesan gambar
def preprocess_image(image):
    image = cv2.resize(image, (320, 320))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Fungsi untuk membuat prediksi
def make_predictions(model, image):
    bbox_pred, class_pred = model.predict(image)
    class_pred = np.argmax(class_pred, axis=-1)
    return bbox_pred, class_pred

# Fungsi untuk menampilkan hasil pada frame
def display_predictions(frame, bbox_pred, class_pred, classes_list):
    if class_pred[0] < len(classes_list):
        label = classes_list[class_pred[0]]
    else:
        label = "Unknown"

    bbox = bbox_pred[0]
    xmin, ymin, xmax, ymax = bbox

    # Menggambar bounding box dan label pada frame
    cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    cv2.putText(frame, label, (int(xmin), int(ymin)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Menggunakan webcam untuk menangkap frame secara real-time
cap = cv2.VideoCapture(0)  # 0 untuk kamera default, ubah jika menggunakan kamera eksternal

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Pra-pemrosesan gambar
    preprocessed_image = preprocess_image(frame)

    # Buat prediksi
    bbox_pred, class_pred = make_predictions(model, preprocessed_image)

    # Tampilkan hasil pada frame
    display_predictions(frame, bbox_pred, class_pred, classes_list)

    # Tampilkan frame dengan prediksi
    cv2.imshow('Real-time Object Detection', frame)

    # Keluar dari loop jika 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Lepaskan kamera dan tutup semua jendela
cap.release()
cv2.destroyAllWindows()